### Дилемма смещения и разброса. Полиномиальные признаки. Регуляризация

In [7]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
from sklearn import linear_model #линейные модели
from sklearn import metrics #метрики
from sklearn import preprocessing # полиномиальные признаки
%matplotlib inline
plt.style.use('seaborn-v0_8')

column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
boston_data = pd.read_csv('data/housing.csv', header=None, delimiter=r"\s+", names=column_names)
boston_data.head()

#Составляем список факторов (исключили целевой столбец)
features = boston_data.drop('MEDV', axis=1).columns
#Составляем матрицу наблюдений X и вектор ответов y
X = boston_data[features]
y = boston_data['MEDV']

Разделим дата-сет на тренировочную и тестовую выборки в соотношении 70/30

In [8]:
# инициализируем функцию по разделению дата сета на тренировочную и тестовую выборки
from sklearn.model_selection import train_test_split #as tts

# Делим выборку 70/30 и установим рандом - 40
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)

# вернем результирующие размеры таблиц
print('Train:', X_train.shape, y_train.shape)
print('Train:', X_test.shape, y_test.shape)

Train: (354, 13) (354,)
Train: (152, 13) (152,)


Обучим линейную регрессию (с помощью МНК) на тренировочных данных и рассчитаем $R^2$ для тренировочных и тестовых данных

In [9]:
# создаем объект класса LinearRegression
lr_model = linear_model.LinearRegression()
# обучаем модель МНК
lr_model.fit(X_train, y_train)

# делаем предсказание для тренировочной выборки
y_train_predict = lr_model.predict(X_train)
# делаем предсказание для тестовой выборки
y_test_predict = lr_model.predict(X_test)

# рассчитываем коэфф. детерминации R2
print('R2 score: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict)))
print('R2 score: {:.3f}'.format(metrics.r2_score(y_test, y_test_predict)))

R2 score: 0.743
R2 score: 0.722


На тренировочной выборке и $R^2$ на тестовой выборке. То есть показатели довольно близки друг к другу (низкий разброс ответов модели для разных выборок). Модель не переучена - no variance

### Регуляризация L1 и L2

In [10]:
# стандартизируем даные
scaler=preprocessing.StandardScaler()

# подгоняем параметры стандартизатора (вычисляем среднее и СКО)
scaler.fit(X_train)

#производим стандартизацию тренировочной выборки
X_train_scaled = scaler.transform(X_train)

# производим стнадартизацию тестовой выборке с таким же параметрами, что и на тренировочной
X_test_scaled = scaler.transform(X_test)

# создаём генератор полиномиальных признаков
poly = preprocessing.PolynomialFeatures(degree=2, include_bias=False)
poly.fit(X_train_scaled)

# генерируем полиномиальные признаки для тренировочной выборки
X_train_scaled_poly = poly.transform(X_train_scaled)
# генерируем полиномиальные признаки для тестовой выборки
X_test_scaled_poly = poly.transform(X_test_scaled)

# выводим результирующие размерности таблиц
print(X_train_scaled_poly.shape)
print(X_test_scaled_poly.shape)

(354, 104)
(152, 104)


#### В sklearn методы регуляризации реализованы в классах:
#### Lasso (L1-регуляризация) и Ridge (L2-регуляризация).
Оба метода осуществляют поиск параметров с добавлением регуляризации. Процесс обучения и предсказания не отличается от обычной линейной регрессии.
<br>Главный параметр инициализации Lasso — это alpha, коэффициент регуляризации. По умолчанию alpha=1. Практика показывает, что это довольно сильная регуляризация для L1-метода. Давайте установим значение этого параметра на 0.1.

In [11]:
# модель линейной регрессии с L1-регуляризацией

# создаём объект класса линейной регрессии L1-регуляризация
lasso_lr_poly = linear_model.Lasso(alpha=0.1)

# обучаем модель
lasso_lr_poly.fit(X_train_scaled_poly, y_train)

# предсказания для тренировочной выборки
y_train_predict_poly = lasso_lr_poly.predict(X_train_scaled_poly)

# предсказания для тестовой выборки
y_test_predict_poly = lasso_lr_poly.predict(X_test_scaled_poly)

# рассчитываем коэфф детерминации R2 для двух выборок
print('R2 score: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict_poly)))
print('R2 score: {:.3f}'.format(metrics.r2_score(y_test, y_test_predict_poly)))

R2 score: 0.879
R2 score: 0.882


Обратите внимание на то, как изменились значения метрик. Да, на тренировочной выборке $R^2 = 0.879$. Метрика упала (до стандартизации + регуляризации значение $R^2$ было $0.929$).
<br>Однако метрика ощутимо выросла на тестовой выборке: $R^2$ = $0.882$ (ранее она была равна $0.268$).
<br>Мы смогли преодолеть переобучение.

In [12]:
# выведем значения коэфф модели, округлив их о третьего знака
print(np.round(lasso_lr_poly.coef_, 3))

[-0.     0.    -0.038  0.    -0.523  2.766 -0.355 -0.605  0.    -0.595
 -0.763  0.    -3.259 -0.    -0.     0.     3.132 -0.141  0.     0.
  0.    -0.     0.     0.    -0.015 -0.     0.063 -0.    -0.     0.
  0.159 -0.    -0.    -0.     0.     0.07  -0.    -0.     0.017  0.
  0.    -0.     0.     0.     0.     0.    -0.    -0.     0.     0.46
 -0.808 -0.643  0.    -0.    -0.     0.    -0.     0.    -0.43  -0.348
 -0.511 -0.     0.    -0.14  -0.    -0.277  0.    -0.     0.223 -0.
 -0.    -0.836 -0.054 -0.421  0.019 -0.784  0.    -0.     0.706  0.
 -0.    -0.335 -0.198  0.    -0.     0.     0.205 -0.     0.531 -0.
  0.     0.048 -0.    -0.292  0.677  0.81  -0.    -1.151 -0.    -0.
 -0.    -0.288 -0.356  0.429]


Большая часть коэффициентов обнулилась. Это значит, что признаки, которые соответствуют этим коэффициентам, не используются в прогнозе модели Lasso-регрессии.